In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 256, 3)
        self.conv3 = nn.Conv2d(256, 512, 3)

        x = torch.randn(28,28).view(-1,1,28,28)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 50)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = np.prod(x[0].shape)
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [4]:
def create_X_y(file):
    X = []
    y = []
    for label, img in np.load(file, allow_pickle=True):
        y.append(torch.tensor(int(label)-1))
        X.append(torch.tensor(img))
    return torch.stack(X), torch.stack(y)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, train_X, train_y):
    BATCH_SIZE = 500
    EPOCHS = 3
    optimizer = optim.SGD(cnn.parameters(), lr=0.1)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 28, 28).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)

            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            
            cnn.zero_grad()
            loss.backward()
            optimizer.step()
            
        print("Epoch: {} Loss: {}".format(epoch+1, loss))


def test(cnn, test_X, test_y):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = cnn(test_X[i].view(-1, 1, 28, 28).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [6]:
train_X, train_y = create_X_y('train.npy')

In [21]:
cnn = CNN().to(device)
train(cnn, train_X, train_y)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [07:07<00:00,  8.73it/s]


Epoch: 1 Loss: 3.8704867362976074


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [07:09<00:00,  8.70it/s]


Epoch: 2 Loss: 3.8527770042419434


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [07:08<00:00,  8.72it/s]


Epoch: 3 Loss: 3.851999044418335


In [14]:
test_X, test_y = create_X_y('test.npy')

In [22]:
test(cnn, test_X.to(device), test_y.to(device))

RuntimeError: CUDA out of memory. Tried to allocate 2.73 GiB (GPU 0; 4.00 GiB total capacity; 26.33 MiB already allocated; 0 bytes free; 2.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF